In [1]:
import pandas as pd
df=pd.read_csv('/content/Dataset-SA.csv')
print(df.head())

                                        product_name product_price Rate  \
0  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
1  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
2  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    3   
3  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    1   
4  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    3   

            Review                                            Summary  \
0           super!  great cooler excellent air flow and for this p...   
1          awesome              best budget 2 fit cooler nice cooling   
2             fair  the quality is good but the power of air is de...   
3  useless product                  very bad product its a only a fan   
4             fair                                      ok ok product   

  Sentiment  
0  positive  
1  positive  
2  positive  
3  negative  
4   neutral  


In [4]:
reviews_df=df[['Review','Sentiment']]
reviews_df.to_csv('review.csv', index=False)
print(reviews_df.head())

            Review Sentiment
0           super!  positive
1          awesome  positive
2             fair  positive
3  useless product  negative
4             fair   neutral


# Hugging Face **transformers**

In [64]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [65]:
import pandas as pd
from transformers import pipeline


In [67]:

classifier=pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


In [76]:
def get_sentiment(review):
    result=classifier(review)
    label=result[0]['label']
    if label=='LABEL_0':
        return 'negative'
    elif label=='LABEL_1':
        return 'neutral'
    else:
        return 'positive'
reviews_df['pred_sentiment']=reviews_df['Review'].apply(get_sentiment)
print(reviews_df)


               Review Sentiment      Cleaned_Review pred_sentiment
0              super!  positive               super       positive
1             awesome  positive             awesome       positive
2                fair  positive                fair        neutral
3     useless product  negative     useless product       negative
4                fair   neutral                fair        neutral
5             awesome  positive             awesome       positive
6  highly recommended  positive  highly recommended       positive
7                nice  positive                nice       positive
8      unsatisfactory  negative      unsatisfactory       negative
9     worth the money  positive     worth the money        neutral


In [86]:
input= "I love this product! It's amazing and works perfectly."
result=classifier(input)
print(result)

[{'label': 'LABEL_2', 'score': 0.9925133585929871}]


# **OpenAPI**

In [42]:
import re

# Function to clean text
def clean_text(text):
    if not isinstance(text, str):
        return ""  # Return an empty string if the input is not a string

    # Remove special characters and punctuation
    text = re.sub(r"[^\w\s]", " ", text)

    # Remove single characters
    text = re.sub(r"\b[a-zA-Z]\b", " ", text)

    # Remove HTML tags
    text = re.sub(r"<[^>]*>", " ", text)

    # Lowercase the text
    text = text.lower()

    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text)

    # Trim leading and trailing spaces
    text = text.strip()

    return text

# Clean the text in the Review column of reviews_df
cleaned_reviews = [clean_text(review) for review in reviews_df['Review']]

# Add the cleaned reviews as a new column to the reviews_df DataFrame
reviews_df['Cleaned_Review'] = cleaned_reviews


<ipython-input-42-1f459a9c37ac>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['Cleaned_Review'] = cleaned_reviews


In [44]:
print(reviews_df.head())

            Review Sentiment   Cleaned_Review
0           super!  positive            super
1          awesome  positive          awesome
2             fair  positive             fair
3  useless product  negative  useless product
4             fair   neutral             fair


In [45]:
total_rows = len(reviews_df)
print(total_rows)
train_size = int(total_rows * 0.8)
test_size = total_rows - train_size
train_set = reviews_df.sample(train_size, random_state=42)
test_set = reviews_df.drop(train_set.index)
print(f"Training set size: {len(train_set)} rows")
print(f"Test set size: {len(test_set)} rows")

205052
Training set size: 164041 rows
Test set size: 41011 rows


In [46]:
!pip install openai==0.27.0

In [77]:
import openai

# Directly assign your API key
openai.api_key = APIKEY




In [78]:
def get_completion(prompt, model="gpt-3.5-turbo-1106"):
    # Sending the prompt to the OpenAI API
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0)

    # Correctly accessing the response
    return response.choices[0]['message']['content']


In [79]:
test_set.shape

(41011, 3)

In [80]:
test_set_total = test_set.sample(100)

test_set_total['pred_label'] = ''

test_set_total

,Review,Sentiment,Cleaned_Review,pred_label
48373,NaN,positive,,
53535,NaN,positive,,
152499,brilliant,positive,brilliant,
77264,NaN,negative,,
129350,waste of money!,negative,waste of money,
...,...,...,...,...
146712,just wow!,positive,just wow,
62607,must buy!,positive,must buy,
134410,worth every penny,positive,worth every penny,
70880,super!,positive,super,


In [81]:
# Assuming `test_set_total` is your DataFrame containing the full dataset
batch_size = 50
batches = []

# Divide your DataFrame into batches
for i in range(0, len(test_set_total), batch_size):
    batch = test_set_total.iloc[i:i + batch_size]  # Use iloc to slice rows
    batches.append(batch)  # Append the batch to the list of batches
 # Append batches instead of assigning

In [82]:
import time

def gpt_completion_function(batch, current_batch, total_batch, model="gpt-3.5-turbo-1106"):
    """Function works in three steps:
    # Step-1: Convert the DataFrame to JSON using the to_json() method.
    # Step-2: Preparing the Gemini Prompt
    # Step-3: Calling GPT API
    """

    print(f"Now processing batch#: {current_batch + 1} of {total_batch}")

    # Convert the batch DataFrame to JSON format
    json_data = batch[['Cleaned_Review', 'pred_label']].to_json(orient='records')

    # Preparing the prompt
    prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
    Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
    Customer reviews are provided between three backticks below.
    In your output, only return the Json code back as output - which is provided between three backticks.
    Your task is to update predicted labels under 'pred_label' in the Json code.
    Don't make any changes to Json code format, please.
    Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

    ```
    {json_data}
    ```
    """

    print(prompt)

    # Send the prompt to the GPT model
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0)

    time.sleep(5)  # Sleep to avoid hitting API rate limits
    return response.choices[0].message["content"]


In [83]:
responses = []

# Get the total number of batches
batch_count = len(batches)

# Loop over each batch and call the GPT function
for i in range(batch_count):
    responses.append(gpt_completion_function(batches[i], i, batch_count))


Now processing batch#: 1 of 2
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
    Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
    Customer reviews are provided between three backticks below.
    In your output, only return the Json code back as output - which is provided between three backticks.
    Your task is to update predicted labels under 'pred_label' in the Json code.
    Don't make any changes to Json code format, please.
    Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

    ```
    [{"Cleaned_Review":"","pred_label":""},{"Cleaned_Review":"","pred_label":""},{"Cleaned_Review":"brilliant","pred_label":""},{"Cleaned_Review":"","pred_label":""},{"Cleaned_Review":"waste of money","pred_label":""},{"Cleaned_Review":"great product","pred_label":""},{"Cleaned_Review":"","pred_label":""},{"Cleaned_

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.